In [ ]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D,Conv1D, Flatten, MaxPooling2D,MaxPooling1D, Reshape
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten
import pickle
from keras.optimizers import Adam
import csv
from sklearn.metrics import confusion_matrix
import time
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout, GRU, LSTM, Bidirectional, Flatten, Dense

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/traffic/tfidf_embeddings.csv")
data

,00pm,10a,10th,125th,12pm,145th,14th,16th,180th,19th,...,yes,yet,york,you,youd,your,youre,youth,zone,class
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,2
2996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0
2997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,1
2998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.184964,0.0,0.200696,0.0,0.0,0.0,0


In [ ]:
X = data.iloc[:, :-1]
y = data.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.60, test_size = 0.40, shuffle=False, random_state = 0)

In [ ]:
# Convert labels to categorical format
NoClass = 3  # Number of classes
y_train_categorical = to_categorical(y_train, num_classes=NoClass)
y_test_categorical = to_categorical(y_test, num_classes=NoClass)


In [ ]:
# Reshape the data for the Conv2D layer
X_train_reshaped = X_train.values.reshape(-1, X_train.shape[1], 1, 1)  # Assuming your data is 2D (samples, features)
X_test_reshaped = X_test.values.reshape(-1, X_test.shape[1], 1, 1)

In [ ]:
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(GRU(64, return_sequences=True))
model.add(Dropout(0.3))
model.add(GRU(32, return_sequences=True))
model.add(Dropout(0.2))

# Adding Bidirectional LSTM
model.add(Bidirectional(LSTM(units=64, return_sequences=True)))

model.add(Flatten())
model.add(Dense(NoClass, activation='softmax'))

In [ ]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 998, 32)           128       
                                                                 
 max_pooling1d (MaxPooling1  (None, 499, 32)           0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 499, 32)           0         
                                                                 
 gru (GRU)                   (None, 499, 64)           18816     
                                                                 
 dropout_1 (Dropout)         (None, 499, 64)           0         
                                                                 
 gru_1 (GRU)                 (None, 499, 32)           9408      
                                                        

In [ ]:
optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model
model.fit(X_train_reshaped, y_train_categorical, epochs=50, batch_size=64, validation_data=(X_test_reshaped, y_test_categorical), verbose=1)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test_categorical)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Epoch 1/50
29/29 [==============================] - 26s 128ms/step - loss: 0.9282 - accuracy: 0.5400 - val_loss: 0.7060 - val_accuracy: 0.6683
Epoch 2/50
29/29 [==============================] - 3s 88ms/step - loss: 0.6140 - accuracy: 0.7494 - val_loss: 0.5317 - val_accuracy: 0.7967
Epoch 3/50
29/29 [==============================] - 3s 96ms/step - loss: 0.4413 - accuracy: 0.8439 - val_loss: 0.3680 - val_accuracy: 0.8575
Epoch 4/50
29/29 [==============================] - 2s 80ms/step - loss: 0.3467 - accuracy: 0.8828 - val_loss: 0.3403 - val_accuracy: 0.8700
Epoch 5/50
29/29 [==============================] - 2s 72ms/step - loss: 0.2783 - accuracy: 0.9044 - val_loss: 0.2827 - val_accuracy: 0.8933
Epoch 6/50
29/29 [==============================] - 2s 79ms/step - loss: 0.2440 - accuracy: 0.9211 - val_loss: 0.3147 - val_accuracy: 0.8808
Epoch 7/50
29/29 [==============================] - 2s 73ms/step - loss: 0.2114 - accuracy: 0.9317 - val_loss: 0.2406 - val_accuracy: 0.9167
Epoch 8/50


In [ ]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test_categorical)
train_loss, train_accuracy = model.evaluate(X_train_reshaped, y_train_categorical)
print(f"Train Accuracy: {train_accuracy * 100:.2f}%")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

57/57 [==============================] - 2s 31ms/step - loss: 0.0016 - accuracy: 0.9994
Train Accuracy: 99.94%
Test Accuracy: 93.08%


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/traffic/tfidf_embeddings.csv")
data

,00pm,10a,10th,125th,12pm,145th,14th,16th,180th,19th,...,yes,yet,york,you,youd,your,youre,youth,zone,class
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,2
2996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0
2997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,1
2998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.184964,0.0,0.200696,0.0,0.0,0.0,0


In [ ]:
X = data.iloc[:, :-1]
y = data.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.60, test_size = 0.40, shuffle=False, random_state = 0)

In [ ]:
# Convert labels to categorical format
NoClass = 3  # Number of classes
y_train_categorical = to_categorical(y_train, num_classes=NoClass)
y_test_categorical = to_categorical(y_test, num_classes=NoClass)


In [ ]:
# Reshape the data for the Conv2D layer
X_train_reshaped = X_train.values.reshape(-1, X_train.shape[1], 1, 1)  # Assuming your data is 2D (samples, features)
X_test_reshaped = X_test.values.reshape(-1, X_test.shape[1], 1, 1)

In [ ]:
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))

# Adding LSTM
model.add(LSTM(units=64, return_sequences=True))

model.add(Dropout(0.3))
model.add(GRU(32, return_sequences=True))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(NoClass, activation='softmax'))

In [ ]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 998, 32)           128       
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 499, 32)           0         
 g1D)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 499, 32)           0         
                                                                 
 lstm_1 (LSTM)               (None, 499, 64)           24832     
                                                                 
 dropout_4 (Dropout)         (None, 499, 64)           0         
                                                                 
 gru_2 (GRU)                 (None, 499, 32)           9408      
                                                      

In [ ]:
optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model
model.fit(X_train_reshaped, y_train_categorical, epochs=50, batch_size=64, validation_data=(X_test_reshaped, y_test_categorical), verbose=1)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test_categorical)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Epoch 1/50
29/29 [==============================] - 6s 66ms/step - loss: 1.0414 - accuracy: 0.4600 - val_loss: 0.8445 - val_accuracy: 0.5967
Epoch 2/50
29/29 [==============================] - 1s 40ms/step - loss: 0.7325 - accuracy: 0.6811 - val_loss: 0.6029 - val_accuracy: 0.7550
Epoch 3/50
29/29 [==============================] - 1s 37ms/step - loss: 0.5856 - accuracy: 0.7711 - val_loss: 0.5018 - val_accuracy: 0.8083
Epoch 4/50
29/29 [==============================] - 1s 52ms/step - loss: 0.4436 - accuracy: 0.8511 - val_loss: 0.4737 - val_accuracy: 0.8308
Epoch 5/50
29/29 [==============================] - 1s 48ms/step - loss: 0.3952 - accuracy: 0.8656 - val_loss: 0.3478 - val_accuracy: 0.8692
Epoch 6/50
29/29 [==============================] - 1s 47ms/step - loss: 0.3260 - accuracy: 0.8917 - val_loss: 0.2997 - val_accuracy: 0.8900
Epoch 7/50
29/29 [==============================] - 2s 76ms/step - loss: 0.2979 - accuracy: 0.8972 - val_loss: 0.2956 - val_accuracy: 0.8942
Epoch 8/50
29

In [ ]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test_categorical)
train_loss, train_accuracy = model.evaluate(X_train_reshaped, y_train_categorical)
print(f"Train Accuracy: {train_accuracy * 100:.2f}%")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

57/57 [==============================] - 1s 13ms/step - loss: 0.0089 - accuracy: 0.9972
Train Accuracy: 99.72%
Test Accuracy: 92.58%


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/traffic/word2vec_embeddings_3.csv")
data

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,class
0,-0.282402,0.266967,0.065208,-0.028991,-0.007023,-0.771427,0.232273,1.207871,-0.591220,-0.309825,...,0.042316,0.310536,0.162041,0.959328,0.440757,0.254530,-0.369880,0.145212,-0.116297,1
1,-0.291088,0.254678,0.064866,-0.057217,-0.062241,-0.777611,0.250659,1.273771,-0.543295,-0.394929,...,0.054725,0.335485,0.092124,0.927354,0.550294,0.228266,-0.344106,0.213569,-0.058613,2
2,-0.261362,0.254425,0.068230,-0.053486,-0.033278,-0.689748,0.213416,1.108689,-0.507155,-0.327768,...,0.065789,0.280096,0.091490,0.850703,0.469683,0.221899,-0.304451,0.158464,-0.077235,1
3,-0.286113,0.391083,0.128172,-0.073136,-0.001999,-0.815213,0.267318,1.340139,-0.703260,-0.387489,...,0.132497,0.319347,0.131601,1.123702,0.524509,0.338532,-0.356903,0.165753,-0.152370,1
4,-0.245734,0.306324,0.101826,-0.071761,-0.020645,-0.699376,0.215515,1.146781,-0.553634,-0.339269,...,0.098094,0.289700,0.093295,0.920155,0.471642,0.250729,-0.294243,0.149158,-0.102090,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,-0.313844,0.486717,0.127441,-0.110789,0.055357,-0.805933,0.282415,1.324012,-0.717741,-0.391278,...,0.213904,0.239798,0.090951,1.214346,0.529893,0.353217,-0.366810,0.117098,-0.188485,2
2996,-0.229370,0.246962,0.072126,-0.027051,-0.002868,-0.601071,0.194752,0.948708,-0.492534,-0.247602,...,0.043127,0.236235,0.106006,0.782842,0.347924,0.208732,-0.286824,0.131814,-0.094364,0
2997,-0.274848,0.296477,0.080733,-0.048536,-0.009738,-0.724364,0.234730,1.157373,-0.573305,-0.327971,...,0.070327,0.281320,0.104546,0.930048,0.461124,0.246414,-0.334507,0.162923,-0.096177,1
2998,-0.233363,0.249164,0.077844,-0.003231,-0.005001,-0.643630,0.205270,0.995505,-0.540571,-0.240271,...,0.018829,0.256943,0.133459,0.824737,0.344565,0.220645,-0.314877,0.141798,-0.104880,0


In [ ]:
X = data.iloc[:, :-1]
y = data.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.60, test_size = 0.40, shuffle=False, random_state = 0)

In [ ]:
# Convert labels to categorical format
NoClass = 3  # Number of classes
y_train_categorical = to_categorical(y_train, num_classes=NoClass)
y_test_categorical = to_categorical(y_test, num_classes=NoClass)


In [ ]:
# Reshape the data for the Conv2D layer
X_train_reshaped = X_train.values.reshape(-1, X_train.shape[1], 1, 1)  # Assuming your data is 2D (samples, features)
X_test_reshaped = X_test.values.reshape(-1, X_test.shape[1], 1, 1)

In [ ]:
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(GRU(64, return_sequences=True))
model.add(Dropout(0.3))
model.add(GRU(32, return_sequences=True))
model.add(Dropout(0.2))

# Adding Bidirectional LSTM
model.add(Bidirectional(LSTM(units=64, return_sequences=True)))

model.add(Flatten())
model.add(Dense(NoClass, activation='softmax'))

In [ ]:
optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model
model.fit(X_train_reshaped, y_train_categorical, epochs=50, batch_size=64, validation_data=(X_test_reshaped, y_test_categorical), verbose=1)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test_categorical)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Epoch 1/50
29/29 [==============================] - 11s 83ms/step - loss: 1.0143 - accuracy: 0.4300 - val_loss: 0.9069 - val_accuracy: 0.5000
Epoch 2/50
29/29 [==============================] - 1s 18ms/step - loss: 0.9486 - accuracy: 0.4911 - val_loss: 0.9716 - val_accuracy: 0.4658
Epoch 3/50
29/29 [==============================] - 0s 17ms/step - loss: 0.9398 - accuracy: 0.5072 - val_loss: 0.9078 - val_accuracy: 0.4967
Epoch 4/50
29/29 [==============================] - 0s 16ms/step - loss: 0.9503 - accuracy: 0.4939 - val_loss: 0.9204 - val_accuracy: 0.5042
Epoch 5/50
29/29 [==============================] - 0s 16ms/step - loss: 0.9270 - accuracy: 0.5206 - val_loss: 0.8832 - val_accuracy: 0.5567
Epoch 6/50
29/29 [==============================] - 0s 16ms/step - loss: 0.9085 - accuracy: 0.5033 - val_loss: 0.8535 - val_accuracy: 0.5733
Epoch 7/50
29/29 [==============================] - 0s 17ms/step - loss: 0.8888 - accuracy: 0.5328 - val_loss: 0.8582 - val_accuracy: 0.5117
Epoch 8/50
2

In [ ]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test_categorical)
train_loss, train_accuracy = model.evaluate(X_train_reshaped, y_train_categorical)
print(f"Train Accuracy: {train_accuracy * 100:.2f}%")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

57/57 [==============================] - 0s 7ms/step - loss: 0.7157 - accuracy: 0.6561
Train Accuracy: 65.61%
Test Accuracy: 66.33%


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/traffic/word2vec_embeddings_3.csv")
data

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,class
0,-0.282402,0.266967,0.065208,-0.028991,-0.007023,-0.771427,0.232273,1.207871,-0.591220,-0.309825,...,0.042316,0.310536,0.162041,0.959328,0.440757,0.254530,-0.369880,0.145212,-0.116297,1
1,-0.291088,0.254678,0.064866,-0.057217,-0.062241,-0.777611,0.250659,1.273771,-0.543295,-0.394929,...,0.054725,0.335485,0.092124,0.927354,0.550294,0.228266,-0.344106,0.213569,-0.058613,2
2,-0.261362,0.254425,0.068230,-0.053486,-0.033278,-0.689748,0.213416,1.108689,-0.507155,-0.327768,...,0.065789,0.280096,0.091490,0.850703,0.469683,0.221899,-0.304451,0.158464,-0.077235,1
3,-0.286113,0.391083,0.128172,-0.073136,-0.001999,-0.815213,0.267318,1.340139,-0.703260,-0.387489,...,0.132497,0.319347,0.131601,1.123702,0.524509,0.338532,-0.356903,0.165753,-0.152370,1
4,-0.245734,0.306324,0.101826,-0.071761,-0.020645,-0.699376,0.215515,1.146781,-0.553634,-0.339269,...,0.098094,0.289700,0.093295,0.920155,0.471642,0.250729,-0.294243,0.149158,-0.102090,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,-0.313844,0.486717,0.127441,-0.110789,0.055357,-0.805933,0.282415,1.324012,-0.717741,-0.391278,...,0.213904,0.239798,0.090951,1.214346,0.529893,0.353217,-0.366810,0.117098,-0.188485,2
2996,-0.229370,0.246962,0.072126,-0.027051,-0.002868,-0.601071,0.194752,0.948708,-0.492534,-0.247602,...,0.043127,0.236235,0.106006,0.782842,0.347924,0.208732,-0.286824,0.131814,-0.094364,0
2997,-0.274848,0.296477,0.080733,-0.048536,-0.009738,-0.724364,0.234730,1.157373,-0.573305,-0.327971,...,0.070327,0.281320,0.104546,0.930048,0.461124,0.246414,-0.334507,0.162923,-0.096177,1
2998,-0.233363,0.249164,0.077844,-0.003231,-0.005001,-0.643630,0.205270,0.995505,-0.540571,-0.240271,...,0.018829,0.256943,0.133459,0.824737,0.344565,0.220645,-0.314877,0.141798,-0.104880,0


In [ ]:
X = data.iloc[:, :-1]
y = data.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.60, test_size = 0.40, shuffle=False, random_state = 0)

In [ ]:
# Convert labels to categorical format
NoClass = 3  # Number of classes
y_train_categorical = to_categorical(y_train, num_classes=NoClass)
y_test_categorical = to_categorical(y_test, num_classes=NoClass)


In [ ]:
# Reshape the data for the Conv2D layer
X_train_reshaped = X_train.values.reshape(-1, X_train.shape[1], 1, 1)  # Assuming your data is 2D (samples, features)
X_test_reshaped = X_test.values.reshape(-1, X_test.shape[1], 1, 1)

In [ ]:
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))

# Adding LSTM
model.add(LSTM(units=64, return_sequences=True))

model.add(Dropout(0.3))
model.add(GRU(32, return_sequences=True))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(NoClass, activation='softmax'))

In [ ]:
optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model
model.fit(X_train_reshaped, y_train_categorical, epochs=50, batch_size=64, validation_data=(X_test_reshaped, y_test_categorical), verbose=1)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test_categorical)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Epoch 1/50
29/29 [==============================] - 6s 38ms/step - loss: 1.0847 - accuracy: 0.3678 - val_loss: 1.0334 - val_accuracy: 0.4717
Epoch 2/50
29/29 [==============================] - 0s 11ms/step - loss: 0.9713 - accuracy: 0.4806 - val_loss: 0.9317 - val_accuracy: 0.5217
Epoch 3/50
29/29 [==============================] - 0s 12ms/step - loss: 0.9375 - accuracy: 0.4944 - val_loss: 0.9134 - val_accuracy: 0.4983
Epoch 4/50
29/29 [==============================] - 0s 14ms/step - loss: 0.9430 - accuracy: 0.5072 - val_loss: 0.9000 - val_accuracy: 0.5592
Epoch 5/50
29/29 [==============================] - 0s 11ms/step - loss: 0.9509 - accuracy: 0.4983 - val_loss: 0.8998 - val_accuracy: 0.5075
Epoch 6/50
29/29 [==============================] - 0s 11ms/step - loss: 0.9328 - accuracy: 0.5033 - val_loss: 0.8968 - val_accuracy: 0.5133
Epoch 7/50
29/29 [==============================] - 0s 12ms/step - loss: 0.9189 - accuracy: 0.4972 - val_loss: 0.8859 - val_accuracy: 0.5383
Epoch 8/50
29

In [ ]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test_categorical)
train_loss, train_accuracy = model.evaluate(X_train_reshaped, y_train_categorical)
print(f"Train Accuracy: {train_accuracy * 100:.2f}%")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

57/57 [==============================] - 0s 4ms/step - loss: 0.7083 - accuracy: 0.6644
Train Accuracy: 66.44%
Test Accuracy: 66.75%
